# Numerical estimate of the optimal CZ gate in hybrid encoding in the KLM model

The target operation in the Fock basis is one that brings $|0010>|01>$ into $-|0010>|01>$, $|0001>|01>$ into $-|0001>|01>$ and acts as the identity on the rest. The operator that describes this transformation is the target $A_t$.

The problem is framed as an optimization problem on the unitary matrix $\Lambda$ which determines the transformation of the bosonic modes. $\Lambda$ has a representation on the space of Fock states $U(\Lambda)$. $U$ acts on the computational states as a set of Kraus operators, of which the first one $A:=K_0$ is of importance as it describes the state obtained after postselection on the ancilla qubits.

The solution consists on optimizing $\Lambda$ on the cost function of $1-F$ where $F$ is the fidelity as defined in [Uskov et al.].

The presence of the hybrid encoding changes the form of the target $A_t$, while the optimization problem remains the same and can be solved as in the previous problem.

## Representation of the unitary on the Fock states

An implementation of the procedure that computes the representation $U$ on the Fock states is given, following [Schee, 2004]

In [1]:
import numpy as np
from math import comb, factorial
from itertools import combinations_with_replacement
from itertools import permutations

def factl(l):
	return [factorial(i) for i in l]

def permanent(matrix):
    n = len(matrix)
    perm_sum = 0

    for perm in permutations(range(n)):
        product = 1
        for i in range(n):
            product *= matrix[i, perm[i]]
        perm_sum += product	

    return perm_sum

def A(lam):
	N = lam.shape[0]
	n = 2
	N_out = comb(N+n-1, n)
	A = np.ones((N_out, N_out))*42

	for idx_n, comb_n in enumerate(combinations_with_replacement(range(N), n)):
		for idx_m, comb_m in enumerate(combinations_with_replacement(range(N), n)):
			n_i = np.zeros(N, dtype=int)
			for x in comb_n:
				n_i[x] += 1
			
			m_i = np.zeros(N, dtype=int)
			for x in comb_m:
				m_i[x] += 1

			omega = []
			for i,x in enumerate(n_i):
				omega.extend([i]*x)
			omega = np.array(omega)

			omegap = []
			for i,x in enumerate(m_i):
				omegap.extend([i]*x)
			omegap = np.array(omegap)

			sub = lam[omegap]
			sub = sub[:, omega]
			prefactor = np.sqrt(np.prod(factl(omega)) * np.prod(factl(omegap)))
			A[idx_m, idx_n] =  prefactor * permanent(sub)
	return A

In [6]:
# example:
lam = np.random.random((3,3))*10
print(lam)
A(lam)

[[1.76189055 1.83984492 3.58338638]
 [5.73657834 2.63543963 4.90343875]
 [4.96939676 2.95100841 0.52825798]]


array([[ 6.20851662,  6.48321075, 17.85737252,  6.77005864, 18.6474671 ,
        51.36263179],
       [20.21444633, 15.19777068, 41.28895359,  9.69760042, 26.1139701 ,
        70.28366251],
       [24.76438759, 20.28303729, 37.47600286, 15.35665609, 23.09303224,
        10.70815619],
       [65.81666203, 30.2368118 , 79.56071487, 13.89108411, 36.55096275,
        96.17484624],
       [80.63091624, 42.46209612, 54.79505191, 21.99725623, 31.72456204,
        14.65284019],
       [98.77961679, 58.65892662, 14.84994111, 34.8338026 ,  8.81843475,
         2.23245198]])

## Target matrix

In [9]:
N = 6 # six photonic modes
n = 2 # two photons
N_fock = comb(N+n-1, n)
At = np.identity(N_fock)

def index_of_fock_state(N, n, v):
	v = np.array(v)
	i = 0
	for c in combinations_with_replacement(range(N), n):
		fock_s = np.zeros(N, dtype=int)
		for x in c:
			fock_s[x] += 1
		if np.all(np.array(fock_s) == v):
			return i
		i += 1

fock_state = [0,0,1,0,0,1]
idx = index_of_fock_state(N, n, fock_state)
At[idx, idx] = -1

fock_state = [0,0,0,1,0,1]
idx = index_of_fock_state(N, n, fock_state)
At[idx, idx] = -1

# Columns outside of the input space allowed by the encoding can be dropped to obtain a representation in the form of [Uskov et al.]
allowed_input_space = np.array([
	[1,0,0,0,1,0],
	[0,1,0,0,1,0],
	[0,0,1,0,1,0],
	[0,0,0,1,1,0],
	[1,0,0,0,0,1],
	[0,1,0,0,0,1],
	[0,0,1,0,0,1],
	[0,0,0,1,0,1]
])
allowed_idx = []
for state in allowed_input_space:
	idx = index_of_fock_state(N, n, state)
	allowed_idx.append(idx)

At = At[:, allowed_idx]

In [11]:
print(At)

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


## Optimization problem

The optimization problem is solved as in the first problem